## Liver like satellite

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.conv_learner import *

In [ ]:
#PATH = 'data/planet/'
#PATH = '/home/farzin/biotech/gdc/tmp/'
PATH = '/DATA/GDC/lana/samples/'

In [ ]:
ls {PATH}

In [ ]:
from fastai.plots import *

In [ ]:
def get_1st(path): return glob(f'{path}/*.*')[0]

In [ ]:
list_paths = [f"{PATH}train/0/TCGA-2Y-A9HA-01A-01-TS1_24.png", f"{PATH}train/9/TCGA-DD-A1E9-01A-02-TSB_13.png"]
titles=["short time till death", "long time till death"]
plots_from_files(list_paths, titles=titles, maintitle="HCC")

In [ ]:
#from planet import f2
#metrics=[f2] #I don't think this is right for our data
f_model = resnet34

We use a different set of data augmentations for this dataset - we also allow vertical flips, since we don't expect vertical orientation of satellite images to change our classifications.

In [ ]:
def get_data(sz):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
    #data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
    return ImageClassifierData.from_paths(PATH, tfms=tfms)

In [ ]:
data = get_data(256) #resize all images to 256 square

Example of pulling/viewing data

In [ ]:
x,y = next(iter(data.val_dl))

In [ ]:
x.shape

In [ ]:
y

In [ ]:
plt.imshow(data.val_ds.denorm(to_np(x))[0]*1.4);

In [ ]:
sz=64

In [ ]:
data = get_data(sz)

In [ ]:
data = data.resize(int(sz*1.3), 'tmp')

In [ ]:
learn = ConvLearner.pretrained(f_model, data)

In [ ]:
lrf=learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 0.3

In [ ]:
learn.fit(lr, 4, cycle_len=1, cycle_mult=2)

In [ ]:
lrs = np.array([lr/9,lr/3,lr])

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=4, cycle_mult=2)

In [ ]:
learn.save(f'{sz}')

In [ ]:
learn.sched.plot_loss()

In [ ]:
sz=128

In [ ]:
learn.set_data(get_data(sz))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(f'{sz}')

In [ ]:
sz=512

In [ ]:
learn.set_data(get_data(sz))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(f'{sz}')

In [ ]:
multi_preds, y = learn.TTA()
preds = np.mean(multi_preds, 0)

In [ ]:
f2(preds,y)

### End